In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from sklearn.model_selection import train_test_split
import os

from utils import *

# Data Wrangling Process Overview
1. Load the data
1. Filter all data to only the intersection of dates across datasets
1. Modify the target dataset to be a binary classification problem (consider 3 classes)
1. Split all the datasets into train, test, and cross-validation sets
1. Melt train, test, and cross-validation sets into tall DataFrames
1. Concatenate columns into train, test, and cross-validation sets for model training

In [2]:
# Load data
data_path = os.path.dirname(os.getcwd()) + '/data'
data_prefix = '/broad_assets_'

target_file = data_path + data_prefix + 'weekly_rets.xlsx'
lag1_file = data_path + data_prefix + 'lag1_rets.xlsx'
roc_file = data_path + data_prefix + 'roc_52w.xlsx'
macd_file = data_path + data_prefix + 'macd.xlsx'
rsi_file = data_path + data_prefix + 'rsi.xlsx'
ewma_file = data_path + data_prefix + 'ewma_vol.xlsx'
skew_file = data_path + data_prefix + 'skew.xlsx'
kurtosis_file = data_path + data_prefix + 'kurtosis.xlsx'

names = ['target', 'lag1', 'roc', 'macd', 'rsi', 'ewma', 'skew', 'kurtosis']
files = [target_file, lag1_file, roc_file, macd_file, rsi_file, ewma_file, skew_file, kurtosis_file]
dfs = {}
for name, file in zip(names, files):
    dfs[name] = pd.read_excel(file, sheet_name=0, index_col=0, parse_dates=True)

In [3]:
dfs['target']

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-07-31,0.015154,0.075677,-0.002266,0.000897,0.042162,0.004952,0.037313,0.002093,-0.002477,-0.008385,-0.031787
2000-08-07,0.067870,0.052623,0.015929,0.013358,0.036514,-0.023689,-0.019612,-0.011556,0.024090,-0.026565,0.034584
2000-08-14,0.015698,0.002262,0.015185,0.006338,0.017116,0.025353,0.029392,0.036872,0.008525,0.006253,0.048137
2000-08-21,0.009488,0.056625,-0.001838,-0.000252,0.006775,-0.026410,0.001619,-0.002437,0.007701,0.001821,0.038370
2000-08-28,0.018391,0.065276,0.001894,0.004126,0.037931,-0.035294,0.019534,-0.017000,0.001856,-0.009469,0.031318
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,0.060689,0.071826,-0.044042,-0.014738,0.020995,0.042517,0.043057,0.088217,0.009326,0.050343,0.058296
2024-08-19,0.100581,0.134610,0.019896,-0.000441,0.103698,0.032142,0.093122,0.077077,-0.023833,0.023948,-0.037742
2024-08-26,0.002016,-0.026628,0.002796,0.006442,0.045003,0.054048,0.029013,-0.017973,-0.019513,0.009670,0.033216


In [4]:
# Get intersection of dates across datasets
dates_inter = reduce(lambda x, y: set(x).intersection(y.index), list(dfs.values())[1:], list(dfs.values())[0].index)
dates_inter = sorted(list(dates_inter))
print(f'Count: {len(dates_inter)}\nFirst: {dates_inter[0]}\nLast:{dates_inter[-1]}')

Count: 1101
First: 2003-08-11 00:00:00
Last:2024-09-06 00:00:00


In [5]:
for name, frame in dfs.items():
    dfs[name] = frame.loc[dates_inter]

In [6]:
dfs['kurtosis']

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2003-08-11,1.079368,2.688712,0.888934,1.483409,0.259313,2.337990,1.859787,0.917809,0.577395,5.057493,1.096865
2003-08-18,1.089578,2.703780,0.997301,1.576087,0.256855,2.343757,1.881484,0.890392,0.574968,4.984124,1.105493
2003-08-25,1.110621,2.732552,0.967243,1.516068,0.254371,2.342283,1.907754,0.889080,0.581897,4.918736,1.122595
2003-08-29,1.125732,2.754722,0.954185,1.498328,0.262161,2.334623,1.915983,0.880414,0.584693,4.792230,1.135538
2003-09-08,1.131257,2.774888,0.951455,1.489093,0.258447,2.317104,1.910290,0.869158,0.567051,4.782962,1.137136
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,10.539355,8.366041,3.532441,3.502978,6.184917,21.289157,9.280967,18.869910,1.617902,5.816363,3.194796
2024-08-19,10.533134,8.357998,3.529323,3.500512,6.181385,21.305832,9.283342,18.875128,1.616072,5.813204,3.195747
2024-08-26,10.537164,8.360061,3.526565,3.498177,6.174220,21.315176,9.287450,18.878442,1.613857,5.814129,3.195334


In [7]:
# MACD dataframe needs to be broken out into 3 separate dataframes
macd_line_cols = [col for col in dfs['macd'].columns if 'MACD_Line' in col]
macd_signal_cols = [col for col in dfs['macd'].columns if 'Signal_Line' in col]
macd_hist_cols = [col for col in dfs['macd'].columns if 'MACD_Histogram' in col]

macd_names = ['macd_line', 'macd_signal', 'macd_histogram']
macd_cols = [macd_line_cols, macd_signal_cols, macd_hist_cols]

for name, cols in zip(macd_names, macd_cols):
    dfs[name] = dfs['macd'].loc[:, cols]

In [8]:
dfs['macd_histogram']

,MACD_Histogram_Asset 1,MACD_Histogram_Asset 2,MACD_Histogram_Asset 3,MACD_Histogram_Asset 4,MACD_Histogram_Asset 5,MACD_Histogram_Asset 6,MACD_Histogram_Asset 7,MACD_Histogram_Asset 8,MACD_Histogram_Asset 9,MACD_Histogram_Asset 10,MACD_Histogram_Asset 11
Date,,,,,,,,,,,
2003-08-11,0.001447,0.001537,0.000738,0.000512,0.001845,-0.000047,0.000983,0.001429,-0.000575,0.002823,-0.000771
2003-08-18,0.001617,0.004833,0.001199,0.000524,0.003610,0.000471,0.000711,0.002690,0.001044,-0.002328,-0.000858
2003-08-25,-0.001063,-0.000594,-0.000498,-0.000367,-0.002353,-0.001500,-0.001722,-0.001128,0.000292,-0.001178,-0.000386
2003-08-29,0.000723,0.000407,0.000242,0.000377,0.000135,0.001050,-0.001728,0.000018,-0.001771,0.001908,0.000255
2003-09-08,0.000314,0.000307,0.000767,0.000936,0.000253,-0.000208,0.001538,0.001165,-0.001409,-0.000923,-0.000505
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,0.002656,0.004004,-0.000655,-0.000304,0.001190,-0.000763,0.001670,0.003185,0.000357,0.002824,0.003812
2024-08-19,0.002546,0.003381,-0.000191,-0.000374,0.004371,0.000569,0.002481,0.002508,-0.000620,0.000944,-0.000787
2024-08-26,-0.001093,-0.002740,-0.000824,-0.000151,0.001935,0.001202,-0.000468,-0.001494,0.000165,-0.000252,0.002481


In [9]:
del dfs['macd']

In [10]:
# Make the target data a binary classification
print('Before:')
display(dfs['target'])
print('After:')
display(pd.DataFrame(np.where(dfs['target'] > 0, 1, 0), columns=dfs['target'].columns, index=dfs['target'].index))

Before:


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2003-08-11,-0.004134,-0.069908,0.003661,0.007570,-0.023806,0.002782,0.003604,0.005542,-0.007746,0.071579,0.020839
2003-08-18,0.040214,0.102236,-0.021921,-0.017539,0.096041,-0.003332,0.039839,0.086728,0.031017,-0.020031,-0.007039
2003-08-25,-0.012095,0.033811,0.008085,-0.006691,0.011468,-0.018136,-0.044093,0.041202,0.030667,0.015487,0.019431
2003-08-29,0.029364,0.053217,0.022728,0.014805,0.056876,0.045605,-0.020637,0.026558,-0.016599,0.079618,0.013701
2003-09-08,0.047726,0.068087,0.007280,0.019577,0.080052,0.046298,0.105618,0.091129,-0.018313,-0.002088,-0.058484
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,0.060689,0.071826,-0.044042,-0.014738,0.020995,0.042517,0.043057,0.088217,0.009326,0.050343,0.058296
2024-08-19,0.100581,0.134610,0.019896,-0.000441,0.103698,0.032142,0.093122,0.077077,-0.023833,0.023948,-0.037742
2024-08-26,0.002016,-0.026628,0.002796,0.006442,0.045003,0.054048,0.029013,-0.017973,-0.019513,0.009670,0.033216


After:


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2003-08-11,0,0,1,1,0,1,1,1,0,1,1
2003-08-18,1,1,0,0,1,0,1,1,1,0,0
2003-08-25,0,1,1,0,1,0,0,1,1,1,1
2003-08-29,1,1,1,1,1,1,0,1,0,1,1
2003-09-08,1,1,1,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,1,1,0,0,1,1,1,1,1,1,1
2024-08-19,1,1,1,0,1,1,1,1,0,1,0
2024-08-26,1,0,1,1,1,1,1,0,0,1,1


In [11]:
dfs['target'] = pd.DataFrame(np.where(dfs['target'] > 0, 1, 0), columns=dfs['target'].columns, index=dfs['target'].index)

In [12]:
# Splitting each frame into train-test sets
train_len = round(len(dfs['target'].index) * 0.8)
test_len = len(dfs['target'].index) - train_len
train_dfs, test_dfs = {}, {}
for name, frame in dfs.items():
    train_dfs[name] = frame.iloc[:train_len]
    test_dfs[name] = frame.iloc[-test_len:]

In [13]:
display(train_dfs['target'].tail(5))
display(test_dfs['target'].head(5))

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,1,1,1,1,1,1,0,0,1,1,0
2020-06-01,1,1,0,1,1,1,1,1,0,1,1
2020-06-08,1,1,0,0,1,1,1,1,0,0,1
2020-06-15,0,0,1,1,0,0,0,0,1,1,0
2020-06-22,1,1,0,0,1,0,1,1,1,1,1


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-06-29,0,0,1,1,0,0,0,0,1,1,0
2020-07-06,1,1,0,0,1,1,1,1,0,1,1
2020-07-13,0,0,1,1,0,0,0,0,0,1,1
2020-07-20,1,1,1,1,1,1,1,1,0,1,1
2020-07-27,0,0,1,1,1,1,1,1,0,1,1


In [14]:
train_dfs.keys()

dict_keys(['target', 'lag1', 'roc', 'rsi', 'ewma', 'skew', 'kurtosis', 'macd_line', 'macd_signal', 'macd_histogram'])

In [15]:
# Exporting Train and Test data in portfolio evaluation format before re-shaping for training
train_path = data_path + '/classifier_train'
test_path = data_path + '/classifier_test'
for name in train_dfs.keys():
    train_dfs[name].to_excel(train_path + f'/broad_assets_{name}_train.xlsx', sheet_name=f'broad_assets_{name}_train')
    test_dfs[name].to_excel(test_path + f'/broad_assets_{name}_test.xlsx', sheet_name=f'broad_assets_{name}_test')

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [16]:
train_df = pd.DataFrame()
for name, frame in train_dfs.items():
    new_col = frame.melt(value_name=name).drop(columns='variable')
    train_df = pd.concat([train_df, new_col],axis=1)

train_df

,target,lag1,roc,rsi,ewma,skew,kurtosis,macd_line,macd_signal,macd_histogram
0,0,-0.026656,0.140818,46.265533,0.047828,0.309292,1.079368,0.000565,-0.000882,0.001447
1,1,-0.004134,0.074442,57.261623,0.045442,0.304377,1.089578,0.002485,0.000868,0.001617
2,0,0.040214,0.068957,72.581668,0.041329,0.305748,1.110621,-0.000416,0.000647,-0.001063
3,1,-0.012095,0.178558,71.977477,0.038089,0.302109,1.125732,0.001419,0.000696,0.000723
4,1,0.029364,0.273776,73.225184,0.039806,0.296824,1.131257,0.001937,0.001623,0.000314
...,...,...,...,...,...,...,...,...,...,...
9686,0,0.078565,-0.467694,65.888653,0.084502,-0.246609,3.087421,0.002784,0.004457,-0.001674
9687,1,-0.000379,-0.400685,69.245564,0.077707,-0.247060,3.085650,0.002859,0.003053,-0.000195
9688,1,0.044419,-0.359235,73.360311,0.073466,-0.247591,3.082737,0.002860,0.003473,-0.000614
9689,0,0.073231,-0.386457,62.118400,0.074588,-0.250078,3.080032,-0.002479,0.000190,-0.002669


In [17]:
test_df = pd.DataFrame()
for name, frame in test_dfs.items():
    new_col = frame.melt(value_name=name).drop(columns='variable')
    test_df = pd.concat([test_df, new_col],axis=1)

test_df

,target,lag1,roc,rsi,ewma,skew,kurtosis,macd_line,macd_signal,macd_histogram
0,0,0.033702,-0.032965,38.716571,0.099559,-0.140854,11.292315,-0.002744,-0.003090,0.000346
1,1,-0.042305,0.040393,62.207613,0.092577,-0.141921,11.286118,0.003178,-0.000351,0.003529
2,0,0.084668,-0.001617,54.223256,0.083020,-0.141565,11.285679,-0.000724,0.000135,-0.000859
3,1,-0.015195,0.081585,76.348437,0.074584,-0.142578,11.288126,0.001616,0.000932,0.000685
4,0,0.062249,0.048354,64.736458,0.067137,-0.142508,11.292278,-0.000973,-0.000164,-0.000809
...,...,...,...,...,...,...,...,...,...,...
2415,1,-0.024322,-0.103947,50.596977,0.050266,-0.328475,3.194796,0.004634,0.000821,0.003812
2416,0,0.058296,-0.122203,50.865190,0.047275,-0.327739,3.195747,0.000313,0.001100,-0.000787
2417,1,-0.037742,-0.112936,66.279484,0.048713,-0.327712,3.195334,0.003796,0.001315,0.002481
2418,0,0.033216,-0.178141,41.282430,0.049945,-0.327083,3.193553,-0.000322,0.001060,-0.001382


In [18]:
train_df.to_excel(data_path + '/broad_assets_classifier_train_data.xlsx', sheet_name='broad_assets_train')
test_df.to_excel(data_path + '/broad_assets_classifier_test_data.xlsx', sheet_name='broad_assets_test')